# Modeling Session 1
This set of exercises are meant to cover many of the topics from Day 1, with callbacks from Opti 101, 201, 202 along the way. You can work along in this notebook, but if you really just want to see the solutions, those can also be found in the GitHub repo.

In [ ]:
%pip install gurobipy

In [ ]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

## Building Trust in Optimization

### Solution pools

In this section, we'll explore solution pools to build trust in a discrete model by
surfacing *multiple, near‑optimal* operating plans. We'll use the same storage + solar
dispatch context from earlier (charging/discharging a battery, buying from the grid, meeting
demand), but we’ll ask Gurobi to return a *set* of high‑quality alternatives.

Goal: Collect the top‑k solutions within a small optimality gap and compare them on
key features (objective, total grid energy, number of charge/discharge switches).

Note than some of this (the problem, and the approaches) should seem familiar...

#### 🔋 Energy Storage Optimization Model

##### Sets
$$
\begin{aligned}
T = \{0, 1, \dots, 11\} \quad \text{(time periods, e.g., hours)}
\end{aligned}
$$

##### Parameters
$$
\begin{aligned}
p_t & : \text{Electricity price in period } t \; [€/MWh] \\
d_t & : \text{Energy demand in period } t \; [\text{MWh}] \\
\eta_{\text{in}}, \eta_{\text{out}} & : \text{Charging and discharging efficiencies} \\
\text{cap} & : \text{Battery capacity } [\text{MWh}] \\
p_{\max} & : \text{Maximum charge/discharge power } [\text{MW}] \\
s_0 & : \text{Initial state of charge } [\text{MWh}]
\end{aligned}
$$

##### Decision Variables
$$
\begin{aligned}
ch_t &\ge 0 && \text{Charging power (MW)} \\
dis_t &\ge 0 && \text{Discharging power (MW)} \\
grid_t &\ge 0 && \text{Power purchased from the grid (MW)} \\
0 \le soc_t &\le \text{cap} && \text{State of charge (MWh)} \\
y_{ch,t} &\in \{0,1\} && \text{1 if charging in period } t \\
y_{dis,t} &\in \{0,1\} && \text{1 if discharging in period } t
\end{aligned}
$$

##### Objective Function
Minimize total grid cost and a small switching penalty:
$$
\begin{aligned}
\min \; Z = \sum_{t \in T} p_t \cdot grid_t \;+\; 10^{-3} \sum_{t \in T} (y_{ch,t} + y_{dis,t})
\end{aligned}
$$


##### Constraints

1. Mutual Exclusivity
$$
\begin{aligned}
y_{ch,t} + y_{dis,t} \le 1 \quad \forall t \in T
\end{aligned}
$$

2. Power Limits
$$
\begin{aligned}
ch_t &\le p_{\max} \, y_{ch,t} \\
dis_t &\le p_{\max} \, y_{dis,t} \quad \forall t \in T
\end{aligned}
$$

3. Energy Balance
$$
\begin{aligned}
grid_t + \eta_{\text{out}} \, dis_t \ge d_t + ch_t \quad \forall t \in T
\end{aligned}
$$

4. State-of-Charge (SoC) Dynamics
$$
\begin{aligned}
soc_0 &= s_0 + \eta_{\text{in}} \, ch_0 - \frac{1}{\eta_{\text{out}}} \, dis_0 \\
soc_t &= soc_{t-1} + \eta_{\text{in}} \, ch_t - \frac{1}{\eta_{\text{out}}} \, dis_t \quad \forall t > 0 \\
&0 \le soc_t \le \text{cap} \quad \forall t \in T
\end{aligned}
$$



In [ ]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Tiny instance (kept small so we can enumerate several pool solutions quickly)
T = range(12)                        # time periods
price = [20,18,16,15,16,18,25,40,45,35,28,22]  # grid price €/MWh
demand = [5,5,5,5,6,6,7,7,7,6,6,5]  # MWh demand each period

eta_in, eta_out = 0.95, 0.95        # round-trip efficiency
cap = 10.0                          # MWh battery capacity
pmax = 4.0                          # MW charge/discharge limit
s0 = 2.0                            # initial state of charge

m = gp.Model("storage_pool")

# Decision variables
ch = m.addVars(T, name="charge", lb=0, ub=pmax)
dis = m.addVars(T, name="discharge", lb=0, ub=pmax)
grid = m.addVars(T, name="grid", lb=0)
soc = m.addVars(T, name="soc", lb=0, ub=cap)
# on/off switching to discourage rapid toggling (makes model MIP)
y_ch = m.addVars(T, vtype=GRB.BINARY, name="y_ch")
y_dis = m.addVars(T, vtype=GRB.BINARY, name="y_dis")

# Logic: can’t charge and discharge simultaneously
m.addConstrs((y_ch[t] + y_dis[t] <= 1 for t in T), name="no_simul")
m.addConstrs((ch[t] <= pmax * y_ch[t] for t in T), name="charge_gate")
m.addConstrs((dis[t] <= pmax * y_dis[t] for t in T), name="discharge_gate")

# Energy balance vs demand
m.addConstrs((grid[t] + dis[t]*eta_out >= demand[t] + ch[t] for t in T), name="balance")

# State of charge dynamics
for t in T:
    if t == 0:
        m.addConstr(soc[t] == s0 + ch[t]*eta_in - dis[t]/eta_out, name=f"soc_{t}")
    else:
        m.addConstr(soc[t] == soc[t-1] + ch[t]*eta_in - dis[t]/eta_out, name=f"soc_{t}")

# Mild cycling preference: penalize on/off changes to diversify plans
switches = gp.quicksum((y_ch[t] + y_dis[t]) for t in T)

# Cost: energy purchased from grid
cost = gp.quicksum(price[t]*grid[t] for t in T)

# Multi-criteria via a single objective (lexico done elsewhere; here we pool around cost)
m.setObjective(cost + 1e-3 * switches, GRB.MINIMIZE)

# Ask for a pool of near‑optimal solutions
m.setParam(GRB.Param.PoolSearchMode, 2)  # systematic search for k‑best
m.setParam(GRB.Param.PoolSolutions, 500)  # keep up to 30
m.setParam(GRB.Param.PoolGap, 0.02)      # within 2% of best found
m.optimize()

# Collect solutions
rows = []
for i in range(m.SolCount):
    m.setParam(GRB.Param.SolutionNumber, i)
    obj = m.PoolObjVal
    total_grid = sum(grid[t].Xn for t in T)
    total_switch = sum(y_ch[t].Xn + y_dis[t].Xn for t in T)
    rows.append({"obj": obj, "grid_MWh": total_grid, "switch_count": total_switch})

sols = pd.DataFrame(rows).sort_values("obj").reset_index(drop=True)

best_obj = m.ObjVal
sols["rel_diff_to_best_pct"] = ((sols["obj"] - best_obj) / abs(best_obj)* 100).round(6).astype(str) + "%"
sols_unique = sols.drop_duplicates().reset_index(drop=True)
sols_unique


How would you explain the differences between these solutions?

### Mulitple scenarios

In this section, we'll evaluate multiple scenarios by changing the *demand* and *price*
profiles and re‑solving the same dispatch model. This mirrors the “what‑if” analysis you saw
earlier: same structure, new data.

Goal: Run three scenarios (Base, High‑Price Peak, Evening‑Heavy Demand) and compare total
grid cost and final state of charge.

In [ ]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

T = range(12)
base_price = [20,18,16,15,16,18,25,40,45,35,28,22]
base_demand = [5,5,5,5,6,6,7,7,7,6,6,5]

scenarios = {
    "Base": (base_price, base_demand),
    "HighPricePeak": ([p*1.2 if 7<=t<=9 else p for t,p in enumerate(base_price)], base_demand),
    "EveningHeavyDemand": (base_price, [d*1.2 if t>=7 else d for t,d in enumerate(base_demand)]),
}

eta_in, eta_out = 0.95, 0.95
cap, pmax, s0 = 10.0, 4.0, 2.0

def solve(price, demand):
    m = gp.Model("storage_scen")
    ch = m.addVars(T, name="charge", lb=0, ub=pmax)
    dis = m.addVars(T, name="discharge", lb=0, ub=pmax)
    grid = m.addVars(T, name="grid", lb=0)
    soc = m.addVars(T, name="soc", lb=0, ub=cap)

    # LP version to keep focus on param changes
    m.addConstrs((grid[t] + dis[t]*eta_out >= demand[t] + ch[t] for t in T), name="balance")
    for t in T:
        if t == 0:
            m.addConstr(soc[t] == s0 + ch[t]*eta_in - dis[t]/eta_out)
        else:
            m.addConstr(soc[t] == soc[t-1] + ch[t]*eta_in - dis[t]/eta_out)

    cost = gp.quicksum(price[t]*grid[t] for t in T)
    m.setObjective(cost, GRB.MINIMIZE)
    m.setParam("OutputFlag", 0)
    m.optimize()

    return float(cost.getValue()), float(soc[max(T)].X)

rows = []
for name, (p, d) in scenarios.items():
    total_cost, final_soc = solve(p, d)
    rows.append({"scenario": name, "total_cost": round(total_cost,2), "final_soc": round(final_soc,2)})

pd.DataFrame(rows).sort_values("total_cost")

### Pi values and sensitivity analysis

In this section, we'll inspect dual values (π) and sensitivity for the *LP* version
of the dispatch model. The dual (shadow) price on a constraint tells you how much the
objective would improve per unit relaxation of that constraint’s RHS - holding the basis fixed.

Goal: Solve the LP, read π for the balance constraints, and identify which periods
are most “binding” (largest |π|). Then, perturb demand slightly and see if the objective
change agrees with π.

In [ ]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

T = range(12)
price = [20,18,16,15,16,18,25,40,45,35,28,22]
demand = [5,5,5,5,6,6,7,7,7,6,6,5]

eta_in, eta_out = 0.95, 0.95
cap, pmax, s0 = 10.0, 4.0, 2.0

# Build LP (no binaries)
m = gp.Model("storage_duals")
ch = m.addVars(T, name="charge", lb=0, ub=pmax)
dis = m.addVars(T, name="discharge", lb=0, ub=pmax)
grid = m.addVars(T, name="grid", lb=0)
soc = m.addVars(T, name="soc", lb=0, ub=cap)

balance = m.addConstrs((grid[t] + dis[t]*eta_out >= demand[t] + ch[t] for t in T), name="balance")
for t in T:
    if t == 0:
        m.addConstr(soc[t] == s0 + ch[t]*eta_in - dis[t]/eta_out, name=f"soc_{t}")
    else:
        m.addConstr(soc[t] == soc[t-1] + ch[t]*eta_in - dis[t]/eta_out, name=f"soc_{t}")

cost = gp.quicksum(price[t]*grid[t] for t in T)
m.setObjective(cost, GRB.MINIMIZE)
m.setParam("OutputFlag", 0)
m.optimize()

# Read duals (Pi) and slacks
rows = []
for t in T:
    con = balance[t]
    rows.append({
        "t": t,
        "pi": con.Pi,
        "slack": con.Slack,
    })

pi_df = pd.DataFrame(rows).sort_values("pi", ascending=False)
print("Top periods by shadow price (higher pi => relaxing balance helps more):")
display(pi_df.head(6))

# Sensitivity check: bump demand at period with largest pi by +0.1 and compare objective change
t_star = int(pi_df.iloc[0]["t"])
bump = 0.1

def solve_with_demand(d):
    m2 = gp.Model("storage_duals_copy")
    ch2 = m2.addVars(T, name="charge", lb=0, ub=pmax)
    dis2 = m2.addVars(T, name="discharge", lb=0, ub=pmax)
    grid2 = m2.addVars(T, name="grid", lb=0)
    soc2 = m2.addVars(T, name="soc", lb=0, ub=cap)
    bal2 = m2.addConstrs((grid2[t] + dis2[t]*eta_out >= d[t] + ch2[t] for t in T))
    for t in T:
        if t == 0:
            m2.addConstr(soc2[t] == s0 + ch2[t]*eta_in - dis2[t]/eta_out)
        else:
            m2.addConstr(soc2[t] == soc2[t-1] + ch2[t]*eta_in - dis2[t]/eta_out)
    cost2 = gp.quicksum(price[t]*grid2[t] for t in T)
    m2.setObjective(cost2, GRB.MINIMIZE)
    m2.setParam("OutputFlag", 0)
    m2.optimize()
    return float(cost2.getValue())

base_obj = float(cost.getValue())
d2 = demand.copy()
d2[t_star] += bump
new_obj = solve_with_demand(d2)

approx_change = pi_df.iloc[0]["pi"] * bump
actual_change = new_obj - base_obj

summary = pd.DataFrame({
    "t*":[t_star],
    "pi":[pi_df.iloc[0]["pi"]],
    "bump":[bump],
    "approx_obj_change (pi*bump)":[approx_change],
    "actual_obj_change":[actual_change],
})
display(summary)
 
# FYI explanation printout, commas only
print(
    f"\nFYI,\n"
    f"- Period {t_star} had the highest shadow price (pi = {pi_df.iloc[0]['pi']:.1f}), meaning extra demand there is most expensive.\n"
    f"- The bump of {bump} MWh is small enough that the linear, pi-based prediction is valid.\n"
    f"- Predicted vs actual change ({approx_change:.2f} vs {actual_change:.2f}) are nearly identical, so we are within the linear sensitivity range.\n"
    f"- Slack = 0 confirms this constraint was binding.\n"
    f"- In plain terms, adding 0.1 MWh at hour {t_star} costs about €{approx_change:.1f}, exactly as the model’s dual value predicted."
)

### Cautionary Tale: When Duals Lie to You
 
Shadow prices (π’s) are powerful, but they only tell the truth *locally*, right around the current optimal solution, assuming nothing else changes.
 
Imagine this:
 
> The model says π = €45 at hour 8,  
> you say “Adding 1 MWh will cost €45!”,  
> you rerun the model, and the cost jumps by €70.
 
**What happened?**
 
- You crossed a kink in the feasible region, a new constraint became binding, so the old π no longer applies.  
- You took too large a step, duals are like instantaneous slopes, not good for big jumps.  
- You have integer logic (on/off decisions), duals only exist for the continuous relaxation, not for the real MIP behavior.  
- Or your scaling or units are inconsistent, a π in kWh means something different than in MWh.
 
**Moral of the story:**  
Duals are great for intuition and small “what-if” nudges, but risky for big decisions.  
They are like weather forecasts, useful for local conditions, unreliable once you cross into another climate.

## Optimization's Impact in Industry: Finance

### Add a cardinality constraint to the basic Markowitz Portfolio Optimization model
Now we'll apply some of the concepts of *discrete decisions* from the session on Optimization in Finance. A great place to start is by testing out the OptiMod for [Mean-Variance Portfolio Optimization](https://gurobi-optimods.readthedocs.io/en/stable/index.html). Then, we'll build out own version of the same model to compare. Finally, we'll change and add to that model, similar to what's shown in the [Gurobi Finance](https://gurobi-finance.readthedocs.io/en/latest/index.html) documentation, to reflect different scenarios.

In [ ]:
# Install the optimods package
%pip -q install gurobi-optimods
import numpy as np

from gurobi_optimods.datasets import load_portfolio
from gurobi_optimods.portfolio import MeanVariancePortfolio

All of the individual OptiMods come with data to be used right out of the box for each pre-built model. Let's load that and briefly see what form the data is in. If you're using an OptiMod, pay attention the format of the data to make sure any new data is in the same form.

In [ ]:
np.set_printoptions(legacy='1.25')

data = load_portfolio()
cov_matrix = data.cov()
mu = data.mean()

In [ ]:
# Show the expected returns for our potential 'assets'
mu

In [ ]:
# Show the covariance matrix
cov_matrix

The *cardinality constraint* we will implement first will just limit the number of positions we can take (i.e. the number of assets to invest in), which we'll initially set to 3. We also see the risk-aversion parameter, `gamma`($\gamma$). The number used for these examples is not specifically set, so feel free to change it. 

In [ ]:
# We can define this risk-aversion coefficient
gamma = 20
# Max positions K
K = 3

# Fill in the above info for expected returns and the covariance, respectively
mvp = MeanVariancePortfolio(mu, cov_matrix)

# If you want to see the logs, set verbose = True
mp = mvp.efficient_portfolio(gamma, max_positions=K, verbose = False)

print(f"The portfolio's expected return is {round(mp.ret,5)}")

# The portfolio's allocation:
mp.x


Now it's time to build the same model ourselves. Some parts of the code will use Gurobi's Python matrix API. Here are some [basic examples](https://support.gurobi.com/hc/en-us/articles/17278438215313-Tutorial-Getting-Started-with-the-Gurobi-Python-API).

As a reminder, this is the problem we want to model.

Let $S$ be the set of assets available, $\mu$ is the expected return, and $\Sigma$ is the covariance matrix. 
$$
\begin{aligned}
\max_{x \in \mathbb{R}^n}\quad 
& \mu^\top x \;-\; \tfrac{\gamma}{2}\, x^\top \Sigma\, x \\
\text{s.t.}\quad 
& \sum_{i \in S} x = 1,\\
& 0 \le x_i \le 1, \quad \forall i \in S
\end{aligned}
$$

In [ ]:
#### Now model it yourself
gamma = 20
l = 0.01
# Create an empty optimization model
m = gp.Model()

# Add variables: x[i] denotes the proportion invested in stock i
# 0 <= x[i] <= 1
x = m.addMVar(len(mu), lb=0, ub=1, name="x")
b = m.addMVar(len(mu), vtype=gp.GRB.BINARY, name="b")

# Budget constraint: all investments sum up to 1
m.addConstr(x.sum() == 1, name="Budget_Constraint")

# Link the continuous x to the binary b
# If x > 0 then b = 1 (also remember the bounds of x)
m.addConstr(x <= b, name="Indicator")

# Minimal position; see formula (2) above
m.addConstr(x >= l * b, name="Minimal_Position")
# Cardinality constraint: at most K positions
cardinality_constr = m.addConstr(b.sum() <= 3, "Cardinality")

# Define objective function: Maximize expected utility
m.setObjective(
    mu.to_numpy() @ x - gamma / 2 * (x @ cov_matrix.to_numpy() @ x), gp.GRB.MAXIMIZE
)
m.optimize()

In [ ]:
s2 = pd.Series(x.X, index= mm.x.index)
s1 = pd.Series(mm.x)
df = pd.concat([s2,s1], axis=1, ignore_index=True)
df.columns = ['OptiMod','Our Model']
df

### Move risk to a constraint
In the models above, the objective represented a utility function that balanced risk and returns. Let's make a couple of changes to the model. 

- Model the risk as constraint, where the objective is now just to maximize return.
- Incorporate fixed and variable transaction costs. 
- Remove the cardinality constraints from before 

Write the formulation where:
$$
\begin{aligned}
&\sigma_0 \space\text{maximal variance for the portfolio} \\
&c  \space\text{is the fixed transaction costs for any asset, relative to total investment value} \\
&l  \space\text{is the lower bound on position size} \\
&f_i \space\text{is the variable transaction fee for each asset relative to total investment value} 
\end{aligned}
$$

$$
\begin{aligned}
\max_{x \in \mathbb{R}^n}\quad &\mu^\top x\\
\text{s.t.}\quad 
&x^\top \Sigma\, x \le \sigma_0\\
& \sum_{i \in S} x_i + c\sum_{i \in S} b_i + \sum_{i \in S} f_ix_i= 1\\
& x_i \le b_i, \quad \forall i \in S \\
& x_i \ge lb_i, \quad \forall i \in S \\
& 0 \le x_i \le 1 \quad \forall i \in S \\
& b_i \in \{0,1\} \quad \forall i \in S
\end{aligned}
$$

Use the data below to write code for the the above model

In [ ]:
data = load_portfolio()
cov_matrix = data.cov()
mu = data.mean()

# Values for the model parameters:
V = 0.0025  # Maximal admissible variance (sigma^2)
l = 0.001  # Minimal position size
c = 0.0001  # Fixed transaction costs
f = 0.001 * np.ones(mu.shape)  # Variable transaction fees

In [ ]:
# Create an empty optimization model
m = gp.Model()

# Add variables: x[i] denotes the proportion invested in stock i
x = m.addMVar(len(mu), lb=0, ub=1, name="x")
# Add variables: b[i]=1 if stock i is held, and b[i]=0 otherwise
b = m.addMVar(len(mu), vtype=gp.GRB.BINARY, name="b")
# Upper bound on variance
m.addConstr(x @ cov_matrix.to_numpy() @ x <= V, name="Variance")
# Force x to 0 if not traded; see formula (1) above
m.addConstr(x <= b, name="Indicator")
# Minimal position; see formula (2) above
m.addConstr(x >= l * b, name="Minimal_Position")
# Budget constraint: all investments, costs, and fees sum up to 1
budget_constr = m.addConstr(
    x.sum() + b.sum() + f @ x == 1, name="Budget_Constraint"
)
# Define objective: Maximize expected return
m.setObjective(mu.to_numpy() @ x, gp.GRB.MAXIMIZE)

m.optimize()

In [ ]:
pd.Series(x.X)

## Interpreting Log Files

Here is a somewhat random variation of the `quadratic assignment problem`. Here is a formulation, but we really won't need it but it's here for fun. 

#### Mathematical Formulation

##### Sets and Indices
- $i, k \in \{0, 1, \ldots, n-1\}$: Facilities
- $j, l \in \{0, 1, \ldots, n-1\}$: Locations

##### Parameters
- $f_{ik}$: Flow between facility $i$ and facility $k$ (represented as `flow[i][k]`)
- $d_{jl}$: Distance between location $j$ and location $l$ (represented as `dist[j][l]`)
- $n$: Number of facilities (and locations)

##### Decision Variables
- $x_{ij} \in \{0, 1\}$: Binary variable equal to 1 if facility $i$ is assigned to location $j$, 0 otherwise

##### Objective Function

$$
\min \sum_{i=0}^{n-1} \sum_{j=0}^{n-1} \sum_{k=0}^{n-1} \sum_{l=0}^{n-1} c_{ik} \cdot f_{ik} \cdot d_{jl} \cdot x_{ij} \cdot x_{kl}
$$

where the coefficient $c_{ik}$ is defined as:

$$
c_{ik} = \begin{cases} 
-1 & \text{if } (i+k) \bmod 2 = 0 \\
1 & \text{if } (i+k) \bmod 2 = 1
\end{cases}
$$

##### Constraints

**Assignment constraints - Each facility to exactly one location:**
$$
\sum_{j=0}^{n-1} x_{ij} = 1 \quad \forall i \in \{0, 1, \ldots, n-1\}
$$

**Assignment constraints - Each location receives exactly one facility:**
$$
\sum_{i=0}^{n-1} x_{ij} = 1 \quad \forall j \in \{0, 1, \ldots, n-1\}
$$

**Binary constraints:**
$$
x_{ij} \in \{0, 1\} \quad \forall i, j \in \{0, 1, \ldots, n-1\}
$$

Use this data and the model below to solve the problem. You may notice it take a little while. 

In [ ]:
n = 10

# Distance between locations
dist = [[abs(j - k) + 1 for k in range(n)] for j in range(n)]

# Flow/weight between facilities
flow = [[(i + k + 1) * ((i - k) ** 2 + 1) for k in range(n)] for i in range(n)]

Here is the model. You shouldn't have to edit anything here. 

In [ ]:
# Quadratic assignment problem
m1 = gp.Model()

# x[i,j] = 1 if facility i is assigned to location j
x = m1.addVars(n, n, vtype=GRB.BINARY, name="x")

# Build objective function
obj = gp.QuadExpr()
for i in range(n):
    for j in range(n):
        for k in range(n):
            for l in range(n):
                coeff = flow[i][k] * dist[j][l]
                # Alternate signs
                if (i + k) % 2 == 0:
                    coeff = -coeff
                obj.addTerms(coeff, x[i, j], x[k, l])

m1.setObjective(obj, GRB.MINIMIZE)

# Each facility is assigned to exactly one location
for i in range(n):
    m1.addConstr(x.sum(i, "*") == 1, name=f"assign_facility[{i}]")

# Each location is assigned exactly one facility
for j in range(n):
    m1.addConstr(x.sum("*", j) == 1, name=f"assign_to_location[{j}]")

Runs this without editing any parameters to see how long it takes. Then, try to recall a parameter mentioned in the log file session that may fit this type of problem. Try searching the [documentation](https://docs.gurobi.com/projects/optimizer/en/current/).

- Hint 1: Look at the type of terms and how one may want to typically simplify these. 
- Hint 2: You'll be told what it is in a couple of minutes. 

In [ ]:
# Set parameters here
# m.Params.ParameterName = value
m1.optimize()

Here's the whole model with the updated parameter. 

In [ ]:
n = 10

# Quadratic assignment problem
m2 = gp.Model()
# Distance between locations
dist = [[abs(j - k) + 1 for k in range(n)] for j in range(n)]

# Flow/weight between facilities
flow = [[(i + k + 1) * ((i - k) ** 2 + 1) for k in range(n)] for i in range(n)]

# x[i,j] = 1 if facility i is assigned to location j
x = m2.addVars(n, n, vtype=GRB.BINARY, name="x")

# Build objective function
obj = gp.QuadExpr()
for i in range(n):
    for j in range(n):
        for k in range(n):
            for l in range(n):
                coeff = flow[i][k] * dist[j][l]
                # Alternate signs
                if (i + k) % 2 == 0:
                    coeff = -coeff
                obj.addTerms(coeff, x[i, j], x[k, l])

m2.setObjective(obj, GRB.MINIMIZE)

# Each facility is assigned to exactly one location
for i in range(n):
    m2.addConstr(x.sum(i, "*") == 1, name=f"assign_facility[{i}]")

# Each location is assigned exactly one facility
for j in range(n):
    m2.addConstr(x.sum("*", j) == 1, name=f"assign_to_location[{j}]")

# Set parameters here
m2.Params.PreQLinearize = 2
presolved2 = m2.presolve()
presolved2.write("pql2.lp")

m2.optimize()